# The material presented in this notebook is for using in Introduction to Deep Learning (ECE 685D) course, Duke University, Fall 2020.

# Convolutional Neural Networks

* ### LeNet5 with MNIST and CIFAR10

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np

## Dataset

## MNIST

In [6]:
batch_size = 100
train_set,test_set,train_loader,test_loader = {},{},{},{}
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.1307,), (0.3081,))])
train_set['mnist'] = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
train_loader['mnist'] = torch.utils.data.DataLoader(train_set['mnist'], batch_size=batch_size, shuffle=True, num_workers=0)
test_set['mnist'] = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
test_loader['mnist'] = torch.utils.data.DataLoader(test_set['mnist'], batch_size=batch_size, shuffle=False, num_workers=0)

## CIFAR10

In [7]:
train_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])
train_set['cifar10'] = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=train_transform)
train_loader['cifar10'] = torch.utils.data.DataLoader(train_set['cifar10'], batch_size=batch_size, shuffle=True, num_workers=0)
test_set['cifar10'] = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=test_transform)
test_loader['cifar10'] = torch.utils.data.DataLoader(test_set['cifar10'], batch_size=batch_size, shuffle=False, num_workers=0)

Files already downloaded and verified
Files already downloaded and verified


### [LeNet](http://yann.lecun.com/exdb/lenet/) is a classic CNN architecture for MNIST dataset proposed in 1990’s 

![LeNet5](https://miro.medium.com/max/2154/1*1TI1aGBZ4dybR6__DI9dzA.png)

## Model architecture for MNIST

In [8]:
class LeNet5(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(LeNet5, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=in_channels, out_channels=6, kernel_size=5, stride=1, padding=2, bias=True)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1, padding=0, bias=True)
        self.fc1   = nn.Linear(in_features=16*5*5, out_features=120) if data_name == 'mnist' else nn.Linear(in_features=16*6*6, out_features=120)
        self.fc2   = nn.Linear(in_features=120, out_features=84)
        self.fc3   = nn.Linear(in_features=84, out_features=out_channels)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

## Define train() and test()

In [9]:
def train(model, device, train_loader, criterion, optimizer, epoch):
    train_loss = 0
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        if batch_idx % (len(train_loader)//2) == 0:
            print('Train({})[{:.0f}%]: Loss: {:.4f}'.format(
                epoch, 100. * batch_idx / len(train_loader), train_loss/(batch_idx+1)))

def test(model, device, test_loader, criterion, epoch):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss = (test_loss*batch_size)/len(test_loader.dataset)
    print('Test({}): Loss: {:.4f}, Accuracy: {:.4f}%'.format(
        epoch, test_loss, 100. * correct / len(test_loader.dataset)))

## Define make_optimizer() and make_scheduler()

In [12]:
def make_optimizer(optimizer_name, model, **kwargs):
    if optimizer_name=='Adam':
        optimizer = optim.Adam(model.parameters(),lr=kwargs['lr'])
    elif optimizer_name=='SGD':
        optimizer = optim.SGD(model.parameters(),lr=kwargs['lr'],momentum=kwargs['momentum'], weight_decay=kwargs['weight_decay'])
    else:
        raise ValueError('Not valid optimizer name')
    return optimizer
    
def make_scheduler(scheduler_name, optimizer, **kwargs):
    if scheduler_name=='MultiStepLR':
        scheduler = optim.lr_scheduler.MultiStepLR(optimizer,milestones=kwargs['milestones'],gamma=kwargs['factor'])
    else:
        raise ValueError('Not valid scheduler name')
    return scheduler

## Define main()

In [19]:
seed = 1
device = 'cuda'
data_name = 'mnist'
optimizer_name = 'SGD'
scheduler_name = 'MultiStepLR'
num_epochs = 10
lr = 0.05
device = torch.device(device)
torch.manual_seed(1)
torch.cuda.manual_seed(1)
in_channels = 1 if data_name== 'mnist' else 3
out_channels = 10
model = LeNet5(in_channels, out_channels).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = make_optimizer(optimizer_name, model, lr=lr, momentum=0, weight_decay=0)
scheduler = make_scheduler(scheduler_name, optimizer, milestones=[5], factor=0.1)
for epoch in range(1, num_epochs + 1):
    train(model, device, train_loader[data_name], criterion, optimizer, epoch)
    test(model, device, test_loader[data_name], criterion, epoch)
    scheduler.step()
    print('Optimizer Learning rate: {0:.4f}'.format(optimizer.param_groups[0]['lr']))

Train(1)[0%]: Loss: 2.3096
Train(1)[50%]: Loss: 0.7897
Test(1): Loss: 0.1018, Accuracy: 96.6400%
Optimizer Learning rate: 0.0500
Train(2)[0%]: Loss: 0.0879
Train(2)[50%]: Loss: 0.1036
Test(2): Loss: 0.0663, Accuracy: 97.8100%
Optimizer Learning rate: 0.0500
Train(3)[0%]: Loss: 0.1106
Train(3)[50%]: Loss: 0.0662
Test(3): Loss: 0.0474, Accuracy: 98.3700%
Optimizer Learning rate: 0.0500
Train(4)[0%]: Loss: 0.0161
Train(4)[50%]: Loss: 0.0507
Test(4): Loss: 0.0481, Accuracy: 98.3400%
Optimizer Learning rate: 0.0500
Train(5)[0%]: Loss: 0.0109
Train(5)[50%]: Loss: 0.0396
Test(5): Loss: 0.0459, Accuracy: 98.4100%
Optimizer Learning rate: 0.0050
Train(6)[0%]: Loss: 0.0397
Train(6)[50%]: Loss: 0.0272
Test(6): Loss: 0.0310, Accuracy: 98.9300%
Optimizer Learning rate: 0.0050
Train(7)[0%]: Loss: 0.0150
Train(7)[50%]: Loss: 0.0238
Test(7): Loss: 0.0302, Accuracy: 98.9800%
Optimizer Learning rate: 0.0050
Train(8)[0%]: Loss: 0.0138
Train(8)[50%]: Loss: 0.0230
Test(8): Loss: 0.0296, Accuracy: 98.9700%


In [21]:
seed = 1
device = 'cuda'
data_name = 'cifar10'
optimizer_name = 'SGD'
scheduler_name = 'MultiStepLR'
num_epochs = 10
lr = 0.05
device = torch.device(device)
torch.manual_seed(1)
torch.cuda.manual_seed(1)
in_channels = 1 if data_name== 'mnist' else 3
out_channels = 10
model = LeNet5(in_channels, out_channels).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = make_optimizer(optimizer_name, model, lr=lr, momentum=0, weight_decay=0)
scheduler = make_scheduler(scheduler_name, optimizer, milestones=[5], factor=0.1)
for epoch in range(1, num_epochs + 1):
    train(model, device, train_loader[data_name], criterion, optimizer, epoch)
    test(model, device, test_loader[data_name], criterion, epoch)
    scheduler.step()
    print('Optimizer Learning rate: {0:.4f}'.format(optimizer.param_groups[0]['lr']))

Train(1)[0%]: Loss: 2.3027
Train(1)[50%]: Loss: 2.1445
Test(1): Loss: 1.6475, Accuracy: 40.4300%
Optimizer Learning rate: 0.0500
Train(2)[0%]: Loss: 1.7159
Train(2)[50%]: Loss: 1.6845
Test(2): Loss: 1.4809, Accuracy: 46.8100%
Optimizer Learning rate: 0.0500
Train(3)[0%]: Loss: 1.4799
Train(3)[50%]: Loss: 1.5472
Test(3): Loss: 1.4880, Accuracy: 46.2600%
Optimizer Learning rate: 0.0500
Train(4)[0%]: Loss: 1.7816
Train(4)[50%]: Loss: 1.4549
Test(4): Loss: 1.3182, Accuracy: 53.0100%
Optimizer Learning rate: 0.0500
Train(5)[0%]: Loss: 1.4507
Train(5)[50%]: Loss: 1.3901
Test(5): Loss: 1.2838, Accuracy: 54.8400%
Optimizer Learning rate: 0.0050
Train(6)[0%]: Loss: 1.4016
Train(6)[50%]: Loss: 1.2645
Test(6): Loss: 1.2009, Accuracy: 57.6500%
Optimizer Learning rate: 0.0050
Train(7)[0%]: Loss: 1.1127
Train(7)[50%]: Loss: 1.2356
Test(7): Loss: 1.1921, Accuracy: 58.2100%
Optimizer Learning rate: 0.0050
Train(8)[0%]: Loss: 1.1692
Train(8)[50%]: Loss: 1.2178
Test(8): Loss: 1.1771, Accuracy: 58.3500%
